In [1]:
import datetime as dt
import pandas as pd
import numpy as np

import os, copy, time
from pathlib import Path

import pandas as pd
import numpy as np

import multiprocessing as mp
from multiprocessing import Pool

In [2]:
class FEncoding(object):
    def __init__(self, n_jobs = 1, chunks = None, path = None):      
        
        self.categor_types = ['object', 'bool', 'int32', 'int64']
        self.numer_types = ['float', 'float32', 'float64']
        self.time_types = ['datetime64[ns]', 'datetime64[ns, tz]'] 
        # What else? https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

        if n_jobs == None:
            self.n_jobs = 1
        elif n_jobs == -1:
            self.n_jobs = mp.cpu_count()
        else:
            self.n_jobs = n_jobs        
        self.chunks = chunks
        self.path = path
    
    def date_replace_(self, X):
        def pars_date(x):
            fmts = ('%Y', '%b %d, %Y','%b %d, %Y','%B %d, %Y','%B %d %Y','%m/%d/%Y','%m/%d/%y','%b %Y','%B%Y','%b %d,%Y', 
                      '%d.%m.%Y', '%Y.%m.%d', '%d-%m-%Y', '%Y-%m-%d %H:%M:%S')
            t = True
            if str(x.dtype) == 'object':
              for fmt in fmts:
                  try:
                      return pd.Series([dt.datetime.strptime(str(x.iloc[i]), fmt) for i in range(len(x))]).apply(lambda q: q.strftime('%m/%d/%Y')).astype('datetime64[ns]')
                      t = False
                      break 
                  except ValueError as err:
                      pass
            if t and (len(str(x.iloc[0])) > 12):
              try:
                  x = x.astype('float')
                  return pd.Series([dt.datetime.fromtimestamp(x.iloc[i]) for i in range(len(x))]).apply(lambda q: q.strftime('%m/%d/%Y')).astype('datetime64[ns]')
              except ValueError as err:
                  pass
        
        for column in X.columns:
            x = pars_date(X[column])
            try: 
              x.nunique()
              X[column] = x
            except AttributeError:
              pass

        return X

    def initialize_types_(self, X):
        # Sometimes categorical feature can be presented with a float type.
        # Sometimes numerical feature can be presented with an int type.  
        # Let's check for that  
        for column in X.columns:
            c_type = str(X[column].dtype) 
            unique_values_X = X[column].unique()        
            if any(c_type == t for t in self.numer_types) & (len(unique_values_X) < 20):
                print('\n {} has type {} and number of unique values: {}, will be considered as a categorical \n'.format(column, c_type, len(unique_values_X)))
                self.categor_columns.append(column)
            elif any(c_type == t for t in self.categor_types) & (len(unique_values_X) > 20):
                print('\n {} has type {} and number of unique values: {}, will be considered as a numerical \n'.format(column, c_type, len(unique_values_X)))
                self.numer_columns.append(column)
            elif any(c_type == t for t in self.categor_types):
                self.categor_columns.append(column)
            elif any(c_type == t for t in self.time_types):
                self.time_columns.append(column)            
            else:
                self.numer_columns.append(column)
        return self.categor_columns, self.numer_columns, self.time_columns
    
    def bucket_numerical_(self, X):
        # TODO: specify or introduce a criterion which columns to bake
        # K-bins discretization based on quantization

        def get_input_keypoints(f_data, n_kps):
            while len(np.quantile(f_data, np.linspace(0.0, 1.0, num=n_kps))) != len(np.unique(np.quantile(f_data, np.linspace(0.0, 1.0, num=n_kps)))):
                n_kps -= 1
            return np.quantile(f_data, np.linspace(0.0, 1.0, num=n_kps))

        if self.columns_to_buck == 'all_numerical':
            self.columns_to_buck = self.numer_columns            
        if type(self.columns_to_buck) != list:
            raise VlaueError('Identify list of columns_to_buck')

        for column in X.columns:
            if any(column == col for col in self.columns_to_buck):
                print('\n {} bucketing ...'.format(column))
                f_X = X[column].values.ravel()
                X[column + '_bucketed'] = np.digitize(f_X, get_input_keypoints(f_X, self.n_bins))
                if self.drop_current:
                    X.drop(columns=[column], inplace=True)
        return X

    def encode_categor_(self, X, method = 'OrdinalEncoder'):    
        if self.method == 'OrdinalEncoder':
            X = X.astype('object').fillna(-1)
            enc = preprocessing.OrdinalEncoder(dtype='int')
            X = pd.DataFrame(enc.fit_transform(X), columns = X.columns)
        if self.method == 'OneHotEncoder':  
            X = X.astype('object')
            X = pd.get_dummies(X)
        return X

    
    def initialize_types(self, X):
        self.categor_columns, self.numer_columns, self.time_columns = [], [], []
        if self.chunks == None:
            self.chunks  = int(len(X.columns)/self.n_jobs)
        return_ = Pool(processes = self.n_jobs).map(self.initialize_types_, 
                                 [X[list(X.columns)[start: start + self.chunks]] for start in range(0, len(X.columns), self.chunks)]
                                 )
        
        for i in range(len(return_)):
            categor_columns, numer_columns, time_columns = return_[i]
            self.categor_columns += categor_columns
            self.numer_columns += numer_columns
            self.time_columns += time_columns
        return {'categor_columns': self.categor_columns,
                           'numer_columns': self.numer_columns,
                           'time_columns': self.time_columns       
         }

    def bucket_numerical(self, X, 
                         n_bins=5, columns_to_buck = 'all_numerical', 
                         drop_current = False):      
        self.n_bins = n_bins
        self.columns_to_buck = columns_to_buck
        self.drop_current = drop_current
        self.initialize_types(X)
        if self.chunks == None:
              self.chunks  = int(len(X.columns)/self.n_jobs)
        X =  pd.concat(Pool(processes = self.n_jobs).map(self.bucket_numerical_, 
                                [X[list(X.columns)[start: start + self.chunks]] for start in range(0, len(X.columns), self.chunks)]
                                ), axis=1)
        if self.path != None:
              X.to_csv(self.path)
        return X

    def encode_categor(self, X, method = 'OrdinalEncoder'):
        self.method = method
        self.initialize_types(X)        
        if self.chunks == None:
            self.chunks  = int(len(X.columns)/self.n_jobs)

        categor_columns = [i for i in self.categor_columns if i in list(X.columns)]
        numer_columns = [i for i in self.numer_columns if i in list(X.columns)]
        
        X_cat =  pd.concat(Pool(processes = self.n_jobs).map(self.encode_categor_, 
                                [X[categor_columns[start: start + self.chunks]] for start in range(0, len(categor_columns), self.chunks)]
                                ), axis=1)

        X = pd.concat([X[numer_columns],X_cat], axis=1)  

        
        if self.path != None:
              X.to_csv(self.path)
        return X

    def date_replace(self, X):
        if self.chunks == None:
            self.chunks  = int(len(X.columns)/self.n_jobs)        
        X =  pd.concat(Pool(processes = self.n_jobs).map(self.date_replace_, 
                                [X[list(X.columns)[start: start + self.chunks]] for start in range(0, len(X.columns), self.chunks)]
                                ), axis=1)  
    
        print('\n time_columns:', self.initialize_types(X)['time_columns']) 
        if self.path != None:
            X.to_csv(self.path)
        return X

In [3]:
X = pd.DataFrame([
'Jan 19, 1990',
'January 19, 1990',
'Jan 19,1990',
'01/19/1990',
'01/19/90',
'1990',
'Jan 1990',
'01.02.2000',
'2000.02.01',
'01-02-2000',
'2111-01-01 12:48:20',
'123',
'abs 123', 
1339521878.04,
'1339521878.04'
]).T
X = X.append(X)
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,"Jan 19, 1990","January 19, 1990","Jan 19,1990",01/19/1990,01/19/90,1990,Jan 1990,01.02.2000,2000.02.01,01-02-2000,2111-01-01 12:48:20,123,abs 123,1.33952e+09,1339521878.04
0,"Jan 19, 1990","January 19, 1990","Jan 19,1990",01/19/1990,01/19/90,1990,Jan 1990,01.02.2000,2000.02.01,01-02-2000,2111-01-01 12:48:20,123,abs 123,1.33952e+09,1339521878.04


In [4]:
fencoding = FEncoding(-1)

In [5]:
fencoding.initialize_types(X)

{'categor_columns': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'numer_columns': [],
 'time_columns': []}

In [6]:
X = fencoding.date_replace(X)
X


 time_columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1990-01-19,1990-01-19,1990-01-19,1990-01-19,1990-01-19,1990-01-01,1990-01-01,2000-02-01,2000-02-01,2000-02-01,2111-01-01,123,abs 123,2012-06-12,2012-06-12
0,1990-01-19,1990-01-19,1990-01-19,1990-01-19,1990-01-19,1990-01-01,1990-01-01,2000-02-01,2000-02-01,2000-02-01,2111-01-01,123,abs 123,2012-06-12,2012-06-12


In [7]:
fencoding.initialize_types(X)

{'categor_columns': [11, 12],
 'numer_columns': [],
 'time_columns': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14]}